In [ ]:
# # Update sklearn to prevent version mismatches
# !pip install sklearn --upgrade

In [ ]:
# # install joblib. This will be used to save your model. 
# # Restart your kernel after installing 
# !pip install joblib

In [1]:
import pandas as pd
import numpy as np

# Set the seed value for the notebook so the results are reproducible
np.random.seed(1)

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

Using TensorFlow backend.


# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [4]:
# Set features. This will also be used as your x values.
X = df.drop('koi_disposition', axis=1)
y = df['koi_disposition']


# Create a Train Test Split

Use `koi_disposition` for the y values

In [5]:
#Transform y values

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)

# Step 2: One-hot encoding
one_hot_y = to_categorical(encoded_y)
one_hot_y

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, one_hot_y, random_state=42)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [ ]:
#all features are numeric ready to scale
#X_train.dtypes

In [8]:
#use MinMaxScaler to scale X values

from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Select a Model and Fit it with Training Data

In [9]:
#using NLP

# create nlp
nlp = Sequential()

nlp.add(Dense(100,activation='relu',input_dim=X_train_scaled.shape[1]))
# Add a second hidden layer
nlp.add(Dense(100, activation='relu'))
# Add output layer
nlp.add(Dense(y_train.shape[1], activation="softmax"))
    
# Compile the nlp using categorical_crossentropy for the loss function, the adam optimizer,
# and add accuracy to the training metrics
nlp.compile(loss="categorical_crossentropy",optimizer="adam", metrics=['accuracy'])

nlp.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               4100      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 14,503
Trainable params: 14,503
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Use the training data to fit (train) the model
nlp.fit(
    X_train_scaled,
    y_train,
    epochs=100,
    shuffle=True,
    verbose=2
)

Train on 5243 samples
Epoch 1/100
5243/5243 - 1s - loss: 0.5533 - accuracy: 0.7109
Epoch 2/100
5243/5243 - 0s - loss: 0.3595 - accuracy: 0.8173
Epoch 3/100
5243/5243 - 0s - loss: 0.3446 - accuracy: 0.8299
Epoch 4/100
5243/5243 - 0s - loss: 0.3380 - accuracy: 0.8310
Epoch 5/100
5243/5243 - 0s - loss: 0.3310 - accuracy: 0.8425
Epoch 6/100
5243/5243 - 0s - loss: 0.3202 - accuracy: 0.8415
Epoch 7/100
5243/5243 - 0s - loss: 0.3197 - accuracy: 0.8432
Epoch 8/100
5243/5243 - 0s - loss: 0.3177 - accuracy: 0.8476
Epoch 9/100
5243/5243 - 0s - loss: 0.3067 - accuracy: 0.8543
Epoch 10/100
5243/5243 - 0s - loss: 0.3052 - accuracy: 0.8594
Epoch 11/100
5243/5243 - 0s - loss: 0.3014 - accuracy: 0.8657
Epoch 12/100
5243/5243 - 0s - loss: 0.3017 - accuracy: 0.8619
Epoch 13/100
5243/5243 - 0s - loss: 0.3005 - accuracy: 0.8625
Epoch 14/100
5243/5243 - 0s - loss: 0.2940 - accuracy: 0.8655
Epoch 15/100
5243/5243 - 0s - loss: 0.2999 - accuracy: 0.8606
Epoch 16/100
5243/5243 - 0s - loss: 0.2936 - accuracy: 0.

In [11]:
# Evaluate the model using the training data
model_loss, model_accuracy = nlp.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1748/1 - 0s - loss: 0.3230 - accuracy: 0.8896
Loss: 0.26981069485164616, Accuracy: 0.8895881175994873


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [12]:
# Function to create model, required for KerasClassifier
#Sequential() will not be able to do Grid Search

def create_model():
    # create model
    model = Sequential()
    model.add(Dense(100,activation='relu',input_dim=X_train_scaled.shape[1]))
    # Add a second hidden layer
    model.add(Dense(100, activation='relu'))
    # Add output layer
    model.add(Dense(y_train.shape[1], activation="softmax"))
    
    # Compile the model using categorical_crossentropy for the loss function, the adam optimizer,
    # and add accuracy to the training metrics
    model.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=['accuracy'])
    return model


In [13]:
from keras.wrappers.scikit_learn import KerasClassifier
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

In [14]:
from sklearn.model_selection import GridSearchCV
# define the grid search parameters
batch_size = [20,40]
epochs = [150, 200]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train_scaled, y_train)

In [15]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.889756 using {'batch_size': 20, 'epochs': 200}
0.883654 (0.005410) with: {'batch_size': 20, 'epochs': 150}
0.889756 (0.009760) with: {'batch_size': 20, 'epochs': 200}
0.889566 (0.008967) with: {'batch_size': 40, 'epochs': 150}
0.882891 (0.004001) with: {'batch_size': 40, 'epochs': 200}
